In [ ]:

from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd

In [ ]:
pwd

In [ ]:
#https://cloud.google.com/docs/authentication/getting-started#command-line

credentials = service_account.Credentials.from_service_account_file(
    "./Bootcamp-BigQuery-1adda4aebc1b.json")
project_id = 'bootcamp-bigquery-290701'

    

In [ ]:

client = bigquery.Client(credentials = credentials,project = project_id)

In [ ]:
query_job = client.query("""
SELECT
fullvisitorId
,visitId
,visitNumber
,visitStartTime
,date
,socialEngagementType
,channelGrouping
,device.browser
,device.operatingSystem
,device.deviceCategory
,geoNetwork.continent
,geoNetwork.subContinent
,geoNetwork.country
,geoNetwork.region
,geoNetwork.metro
,geoNetwork.city
,trafficSource.referralPath
,trafficSource.source
,trafficSource.medium
,trafficSource.isTrueDirect
,totals.bounces
,totals.hits
,totals.newVisits
,totals.pageviews
,totals.sessionQualityDim
,totals.totalTransactionRevenue
,totals.transactionRevenue
,totals.transactions
,totals.visits

FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`
where _table_suffix between '20170501' and '20170831'

order by 1,2 nulls last
   LIMIT 300000 """)

results = query_job.result() # Wait for the job to complete.

In [ ]:
totals_df = results.to_dataframe()

print(totals_df.shape)
totals_df.head()
# Having a lot of issues using pyarrow. I uninstalled the package and am getting things to work

In [ ]:
totals_df = totals_df.fillna(0)
totals_df.isTrueDirect = totals_df.isTrueDirect.astype('bool')
totals_df.bounces = totals_df.bounces.astype('int')
totals_df.newVisits = totals_df.newVisits.astype('int')
totals_df.pageviews = totals_df.pageviews.astype('int')
totals_df.sessionQualityDim = totals_df.sessionQualityDim.astype('int')
totals_df.totalTransactionRevenue = totals_df.totalTransactionRevenue.astype('int64')
totals_df.transactionRevenue = totals_df.transactionRevenue.astype('int64')
totals_df.transactions = totals_df.transactions.astype('int')
totals_df.dtypes

In [ ]:
def char_len(x, fixed_n):
    '''set string x to fixed_n character, prepend with 'xxx' if short'''
    if len(x) > fixed_n: 
        return x[:fixed_n] 
    elif len(x) < fixed_n: 
        return 'x' * (fixed_n - len(x)) + x 
    return x 
totals_df.referralPath = totals_df.referralPath.apply(lambda x: char_len,254)

In [ ]:
my_query = """
SELECT
distinct
fullVisitorID
,date

 FROM
`bigquery-public-data.google_analytics_sample.ga_sessions_*`
 WHERE _TABLE_SUFFIX BETWEEN '20170801' and '20170831'
 ;
"""


query_job = client.query(my_query)

results = query_job.result() # Wait for the job to complete.

In [ ]:
customers_df = results.to_dataframe()
print(customers_df.shape)
customers_df.head()

# Connect to pgAdmin

In [1]:
from sqlalchemy import create_engine
# from config import db_password2
import psycopg2

In [2]:
engine = psycopg2.connect(
    database = "postgres",
    user = "postgres",
    password = "ucb2020FINAL",
    host = "js-ucbfinalprojectdb.c8s02fywjegc.us-east-2.rds.amazonaws.com",
#     host = "dataviz-db.csxrf9ti2aba.us-east-2.rds.amazonaws.com",
    port = 5432)
cursor = engine.cursor()

In [ ]:
#use Pandas to see all data in table
sql = """
SELECT "table_name","column_name", "data_type", "table_schema"
FROM INFORMATION_SCHEMA.COLUMNS
WHERE "table_schema" = 'public'
ORDER BY table_name  
"""
pd.read_sql(sql, con=engine)

# Load customers and then insert them

In [ ]:
sql = """drop table customers_test2;"""

cursor.execute(sql)
sql = """
CREATE TABLE customers_test2 (
     fullvisitorId VARCHAR(19) NOT NULL,
	 date DATE NOT NULL
	 --PRIMARY KEY (fullvisitorId),
);
"""
cursor.execute(sql)
engine.commit()

In [ ]:
from io import StringIO

#use stringIO to initizalize a string buffer so we can "copy" the csv from the buffer into the dB

# Initialize a string buffer
sio = StringIO()
sio.write(customers_df.to_csv(index = None,header = None)) #Write the Pandas df as a csv to the buffer
sio.seek(0) # be sure to reset the position to the start of the stream

# Copy the string buffer into the db as if it were an actual file
with engine.cursor() as c:
    c.copy_from (sio,"customers_test2",columns = customers_df.columns,sep = ',')
    engine.commit()

In [ ]:
sql = """delete from bigquery_totals;"""
cursor.execute(sql)
engine.commit()


In [ ]:
# Initialize a string buffer
sio = StringIO()
sio.write(totals_df.to_csv(index = None,header = None,sep = "|")) #Write the Pandas df as a csv to the buffer
sio.seek(0) # be sure to reset the position to the start of the stream

# Copy the string buffer into the db as if it were an actual file
with engine.cursor() as c:
    c.copy_from (sio,"bigquery_totals",columns = totals_df.columns,sep = '|')
    engine.commit()

In [ ]:
engine.commit()

In [3]:
cursor.execute("select count(*) from bigquery_totals;")
cursor.fetchall()

[(203317,)]

In [ ]:
len(totals_df.referralPath)